## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb　←いまここ

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

## 課題と機関と研究代表者をつなぐ

In [ ]:
# encoding: utf-8
import pandas as pd
import numpy as np
import pickle

### 課題を読み込む

In [ ]:
with open('afterCleaning/parse_kadai.dump', 'rb') as f:
    kadai = pickle.load(f)

In [ ]:
kadai

In [ ]:
kadai = kadai[['category_niicode', 'section_niicode','startfiscalyear', 'directcost']]
kadai

### 研究機関を読み込む

In [ ]:
with open('afterCleaning/parse_institution_from_grantlist.dump', 'rb') as f:
    institution = pickle.load(f)

In [ ]:
institution

In [ ]:
institution = institution[['institution_niicode']]
institution

### 研究代表者を読み込む

In [ ]:
with open('afterCleaning/parse_member_from_summary.dump', 'rb') as f:
    member = pickle.load(f)

In [ ]:
member

In [ ]:
member = member[['kenkyuusha_id']]
member

### 課題と機関と研究代表者をJOINする

In [ ]:
df = kadai.join(institution)
df = df.join(member)
df

In [ ]:
df.info()

In [ ]:
df = df.fillna(0)
df.info()

In [ ]:
df = df.astype({
    'kenkyuusha_id': np.int64
})
df.info()

In [ ]:
df

ローカルのMariaDBに保存する

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('grantaward', engine, if_exists='replace',
          dtype={
              'awardnumber': String(256),
              'category_niicode': Integer,
              'section': String(256),
              'startfiscalyear': Integer,
              'directcost': Integer,
              'niicode': Integer,
              'kenkyuusha_id': Integer,                 
                })

全データの保存に5秒くらいかかる。無事にコミットできたら終了。あとは分析に使う。